说明：这个notebook演示了如何使用quantification方法（目前实现了4种方法进行隐私量化）
1. dFIL (batchsize = 1)
2. distance correlation (batchsize>=2)
3. mutual information (batchsize>=8)
4. ULoss (batchsize = 1)

注意用不同方法的时候要重新设置 批大小 （即args['batch_size']的值）

因为在整个测试集上进行隐私量化，时间太长了（可能要跑好几天）所以这里设计了一个get_one_data()函数，取测试集的前k个数据作为一个数据集，batch_size=k,因此只需要迭代一次

In [14]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
# os.environ['NUMEXPR_MAX_THREADS'] = '48'

# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.quantification.distance_correlation.distCor import distCorMetric
from ppsplit.quantification.fisher_information.dFIL_inverse import dFILInverseMetric
from ppsplit.quantification.shannon_information.mutual_information import MuInfoMetric
from ppsplit.quantification.shannon_information.ULoss import ULossMetric

# 导入各个baseline模型及其数据集预处理方法
# 模型
from target_model.models.splitnn_utils import split_weights_client
from target_model.models.VGG import VGG,VGG5Decoder,model_cfg
from target_model.models.BankNet import BankNet1,bank_cfg
from target_model.models.CreditNet import CreditNet1,credit_cfg
from target_model.models.PurchaseNet import PurchaseClassifier1,purchase_cfg
# 数据预处理方法
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,get_one_data,deprocess
from target_model.data_preprocessing.preprocess_bank import bank_dataset,preprocess_bank
from target_model.data_preprocessing.preprocess_credit import preprocess_credit
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase

# utils
from ppsplit.utils.utils import create_dir

In [15]:
# 基本参数：
# 硬件
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# 参数
# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type = str, default = 'CIFAR10')
# parser.add_argument('--device', type = str, default = 'cuda:1')
# parser.add_argument('--batch_size',type=int, default=1) # muinfo最小为8，# distcor最小为2
# args = parser.parse_args()

args = {
        'device':torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        # 'dataset':'purchase',
        'batch_size':5,
        'noise_scale':1000, # 防护措施
        }
print(args['device'])

cuda:1


# 数据集及其模型加载

In [17]:
# 加载模型和数据集，并从unit模型中切割出client_model
if args['dataset']=='CIFAR10':
    # 超参数
    testset_len = 10000 # 10000个数据一次 整个测试集合的长度
    # split_layer_list = list(range(len(model_cfg['VGG5'])))
    split_layer = 2 # 定成3吧？
    test_num = 2 # 试验序号

    # 关键路径
    unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/BN+Tanh/VGG5-params-20ep.pth' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
    results_dir  = f'../results/InvMetric-202403/VGG5/quantification/{test_num}/'
    decoder_route = f"../results/InvMetric-202403/VGG5/{test_num}/Decoder-layer{split_layer}.pth"

    # 数据集加载
    trainloader,testloader = get_cifar10_normalize(batch_size = 5)
    one_data_loader = get_one_data(testloader,batch_size = args['batch_size']) #拿到第一个测试数据

    # 切割成client model
    # vgg5_unit.load_state_dict(torch.load(unit_net_route,map_location=torch.device('cpu'))) # 完整的模型
    client_net = VGG('Client','VGG5',split_layer,model_cfg,noise_scale=args['noise_scale'])
    pweights = torch.load(unit_net_route)
    if split_layer < len(model_cfg['VGG5']):
        pweights = split_weights_client(pweights,client_net.state_dict())
    client_net.load_state_dict(pweights)

elif args['dataset']=='bank':
    # 超参数
    test_num = 1 # 试验序号
    testset_len=8238
    # split_layer_list = ['linear1', 'linear2']
    split_layer_list = [0,2,4,6]
    split_layer = 2

    # 关键路径
    results_dir  = f'../results/InvMetric-202403/Bank/quantification/{test_num}/'
    unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/Bank/bank-20ep_params.pth'
    decoder_route = f"../results/InvMetric-202403/Bank/{test_num}/Decoder-layer{split_layer}.pth"

    # 数据集加载
    trainloader,testloader = preprocess_bank(batch_size=1)
    one_data_loader = get_one_data(testloader,batch_size = args['batch_size']) #拿到第一个测试数据 

    # 模型加载
    client_net = BankNet1(layer=split_layer,noise_scale=args['noise_scale'])
    pweights = torch.load(unit_net_route)
    if split_layer < len(bank_cfg):
        pweights = split_weights_client(pweights,client_net.state_dict())
    client_net.load_state_dict(pweights)

elif args['dataset']=='credit':
    # 超参数
    test_num = 1 # 试验序号
    testset_len = 61503 # for the mutual information
    split_layer_list = [0,3,6,9]
    split_layer = 3
    # split_layer_list = ['linear1', 'linear2']

    # 关键路径
    results_dir  = f'../results/InvMetric-202403/Credit/quantification/{test_num}/'
    unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/credit/credit-20ep_params.pth'
    decoder_route = f"../results/InvMetric-202403/Credit/{test_num}/Decoder-layer{split_layer}.pth"

    # 数据集加载
    trainloader,testloader = preprocess_credit(batch_size=1)
    one_data_loader = get_one_data(testloader,batch_size = args['batch_size']) #拿到第一个测试数据

    # client模型切割加载
    client_net = CreditNet1(layer=split_layer,noise_scale=args['noise_scale'])
    pweights = torch.load(unit_net_route)
    if split_layer < len(credit_cfg):
        pweights = split_weights_client(pweights,client_net.state_dict())
    client_net.load_state_dict(pweights)

elif args['dataset']=='purchase':
    # 超参数
    test_num = 1 # 试验序号
    testset_len = 39465 # test len
    # split_layer_list = [0,1,2,3,4,5,6,7,8]
    split_layer = 3

    # 关键路径
    results_dir = f'../results/InvMetric-202403/Purchase/quantification/{test_num}/'
    unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/Purchase100/Purchase_bestmodel_param.pth'
    decoder_route = f"../results/InvMetric-202403/Purchase/{test_num}/Decoder-layer{split_layer}.pth"
    
    # 数据集加载
    trainloader,testloader = preprocess_purchase(batch_size=1)
    one_data_loader = get_one_data(testloader,batch_size = args['batch_size']) #拿到第一个测试数据

    # 模型加载
    client_net = PurchaseClassifier1(layer=split_layer,noise_scale=args['noise_scale'])
    # pweights = torch.load(unit_net_route,map_location=torch.device('cpu'))
    pweights = torch.load(unit_net_route)
    if split_layer < len(purchase_cfg):
        pweights = split_weights_client(pweights,client_net.state_dict())
    client_net.load_state_dict(pweights)

else:
    exit(-1)


features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.running_var
features.1.num_batches_tracked
features.4.weight
features.4.bias
features.5.weight
features.5.bias
features.5.running_mean
features.5.running_var
features.5.num_batches_tracked


In [18]:
# 创建文件夹
create_dir(results_dir)

# client_net使用
client_net = client_net.to(args['device'])
client_net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
  )
  (denses): Sequential()
)

# 各种指标计算

## 1.dFIL-inverse
注意：batchsize 需要等于1

In [6]:
# dFIL inverse指标计算

eta_same_layer_list = []
eta_diff_layer_list=[]

metric = dFILInverseMetric()
# 对traingloader遍历计算所有 inverse dFIL
# for j, data in enumerate(tqdm.tqdm(testloader)):
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    # if j < 31705:
        # continue
    inputs, labels = data
    inputs, labels = inputs.to(args['device']), labels.to(args['device'])
    inputs.requires_grad_(True) # 需要求导
    
    # inference
    outputs = client_net(inputs)

    eta = metric.quantify(model=client_net, inputs=inputs, outputs=outputs, with_outputs=True)
    # 打印
    # print(str(j)+": "+str(eta.item()))
    eta_same_layer_list.append(eta)
eta_diff_layer_list.append(eta_same_layer_list)

# 结果储存到csv中
matrix = np.array(eta_diff_layer_list) # 有点大，x
transpose = matrix.T # 一行一条数据，一列代表一个layer 
# pd.DataFrame(data=transpose, columns=[i for i in split_layer_list]).to_csv(save_img_dir + f'dFIL-1.csv',index=False)
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'dFIL.csv',index=False)



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.57s/it]


## 2. distance correlation
注意：batchsize >=2 

In [20]:
# distance correlation指标计算


distCorr_diff_layer_list = []
distCorr_same_layer_list = []
metric = distCorMetric()

for j, data in enumerate(tqdm.tqdm(testloader)): # 对testloader遍历
# for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    tab, labels = data
    tab, labels = tab.to(args['device']), labels.to(args['device'])
    with torch.no_grad():
        pred = client_net(tab).cpu().detach()
        inputs = tab.cpu().detach()

        distCorr = metric.quantify(inputs=inputs,outputs=pred) # x,z
        distCorr_same_layer_list.append(distCorr)


print(f"Layer {split_layer} Avg distCorr: {sum(distCorr_same_layer_list)/len(distCorr_same_layer_list)}")
distCorr_diff_layer_list.append(distCorr_same_layer_list)

# 保存到csv中
matrix = np.array(distCorr_diff_layer_list) # 有点大，x
transpose = matrix.T # 一行一条数据，一列代表一个layer 
# pd.DataFrame(data=transpose, columns=[i for i in range (len(split_layer_list))]).to_csv(save_img_dir + f'DLoss-bs{batch_size}.csv',index=False)
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'DLoss.csv',index=False)



100%|██████████| 2000/2000 [00:11<00:00, 167.23it/s]


Layer 2 Avg distCorr: 0.9745437641441822


## 3. mutual information
注意：batchsize>=8

In [10]:
# mutual information指标计算

MI_diff_layer_list = []
MI_same_layer_list = []
metric = MuInfoMetric()

# for j, data in enumerate(tqdm.tqdm(testloader)): # 对testloader遍历
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    images, labels = data
    images, labels = images.to(args['device']), labels.to(args['device'])
    with torch.no_grad():
        # inference
        outputs = client_net(images).clone().detach()
        inputs = images.cpu().detach()
        mi = metric.quantify(inputs=inputs, outputs = outputs)
        MI_same_layer_list.append(mi)
        
print(f"Layer {split_layer} MI: {sum(MI_same_layer_list)/len(MI_same_layer_list)}")
MI_diff_layer_list.append(MI_same_layer_list)

# 保存到csv中
matrix = np.array(MI_diff_layer_list) # 有点大，x
transpose = matrix.T # 一行一条数据，一列代表一个layer 
# pd.DataFrame(data=transpose, columns=[i for i in split_layer_list]).to_csv(results_dir + f'MI-bs{batch_size}.csv',index=False)
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'MILoss.csv',index=False)


NameError: name 'one_data_loader' is not defined

## 4. Uncertainty Loss
注意：batchsize=1

In [16]:
# mutual information指标计算


ULoss_diff_layer_list = []
ULoss_same_layer_list = []
metric = ULossMetric()
decoder_net = torch.load(decoder_route)
decoder_net.to(args['device'])
decoder_net.eval()

# for j, data in enumerate(tqdm.tqdm(testloader)): # 对testloader遍历
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    images, labels = data
    images, labels = images.to(args['device']), labels.to(args['device'])
    with torch.no_grad():
        # inference
        outputs = client_net(images).clone().detach()
        uloss = metric.quantify(output = outputs, decoder_net=decoder_net)
        ULoss_same_layer_list.append(uloss)
        
print(f"Layer {split_layer} ULoss: {sum(ULoss_same_layer_list)/len(ULoss_same_layer_list)}")
ULoss_diff_layer_list.append(ULoss_same_layer_list)

# 保存到csv中
matrix = np.array(ULoss_diff_layer_list) # 有点大，x
transpose = matrix.T # 一行一条数据，一列代表一个layer 
# pd.DataFrame(data=transpose, columns=[i for i in split_layer_list]).to_csv(results_dir + f'ULoss-bs{batch_size}.csv',index=False)
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'ULoss.csv',index=False)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  4.55it/s]

Layer 3 ULoss: 4.021282423374509
